# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

from onnxruntime.transformers.models.stable_diffusion.diffusion_models import BaseModel

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [105]:
import chromadb
from dotenv import load_dotenv
from pydantic import BaseModel, Field, ConfigDict
from tavily import TavilyClient

from project.lib.agents import Agent
from project.lib.llm import LLM
from project.lib.messages import UserMessage, SystemMessage, BaseMessage, ToolMessage, AIMessage
from project.lib.tooling import tool
from project.lib.evaluation import EvaluationReport

In [90]:
load_dotenv()

True

In [91]:
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

In [92]:
class Game(BaseModel):
    """Game information"""
    model_config = ConfigDict(
        frozen=True,
    )

    Platform: str = Field(
        ...,
        description="Platform name of the game",
        alias="Platform",
        examples=["Game Boy", "Playstation 5", "Xbox 360"]
    )
    Name: str = Field(
        ...,
        description="Name of the Game",
        alias="Name",
        examples=["Super Mario World", "Pokémon Red and Blue", "Gran Turismo"]
    )
    YearOfRelease: int = Field(
        ...,
        description="Year when that game was released for that platform",
        alias="YearOfRelease",
        examples=[1990, 2001, 2010]
    )
    Description: str = Field(
        ...,
        description="Additional details about the game",
        alias="Description",
        examples=["A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser."]
    )

In [93]:
@tool
def retrieve_game(query: str, num_of_results = 5) -> list[Game]:
    """
    Semantic search: Finds most results in the vector DB

    args:
    - query: a question about game industry.
    - num_of_results: number of results to return

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    response = collection.query(query_texts=[query], n_results=num_of_results)

    if not (metadata := response.get("metadatas", [[]])[0]):
        return metadata

    return [Game(**meta) for meta in metadata]

In [94]:
test_question = "When was Fifa 98 released?"
list_of_games = retrieve_game(test_question)
print(list_of_games)
print(test_question)

[Game(Platform='PlayStation 1', Name='Gran Turismo', YearOfRelease=1997, Description='A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.'), Game(Platform='Wii', Name='Wii Sports', YearOfRelease=2006, Description="A collection of sports games that utilize the Wii's motion controls, bundled with the console to showcase its capabilities."), Game(Platform='Nintendo 64', Name='Super Mario 64', YearOfRelease=1996, Description="A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach."), Game(Platform='Game Boy Color', Name='Pokémon Gold and Silver', YearOfRelease=1999, Description='Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.'), Game(Platform='PlayStation 3', Name='Gran Turismo 5', YearOfRelease=2010, Description='A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.')

In [103]:
@tool
def evaluate_retrieval(question: str, retrieved_docs: list[Game]) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.

    args:
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database

    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    if not retrieved_docs:
        return EvaluationReport(
            useful=False,
            description="No documents were retrieved, so it's not possible to answer the question."
        )

    llm_evaluator = LLM()
    messages: list[BaseMessage] = [
        SystemMessage(
            content=(
                "You are an expert in evaluating documents."
                "Your task is to evaluate if the information in the documents are enough to respond the user question."
                "Give a detailed explanation, so it's possible to take an action to accept it or not"
            )
        ),
        UserMessage(
            content=(
                f"""
                    # Question: {question}
                    # Documents: {[doc.model_dump() for doc in retrieved_docs]}
                    """
            )
        )
    ]

    response = llm_evaluator.invoke(messages, response_format=EvaluationReport)

    if response.content:
        return EvaluationReport.model_validate_json(response.content)

    return EvaluationReport(
        useful=False,
        description="We were not able to evaluate the documents. Please try again."
    )

In [104]:
result = evaluate_retrieval(test_question, list_of_games)
print(result)

useful=False description="The documents provided do not contain any information regarding the release date of FIFA 98. They list various video games along with their platforms, names, and years of release, but none of them mention FIFA 98 or its release year. Therefore, the documents are not useful for answering the user's question."


In [115]:
class SearchResult(BaseModel):
    """Search result"""
    model_config = ConfigDict(
        frozen=True,
    )

    title: str = Field(
        ...,
        description="Title of the search result",
        alias="title",
        examples=["Fifa 98"]
    )
    content: str = Field(
        ...,
        description="Content of the search result",
        alias="content",
        examples=["Fifa 98 was released in 1998."]
    )
    score: float = Field(
        default=0.0,
        description="Score of the search result",
        alias="score",
        examples=[0.9]
    )

In [121]:
@tool
def game_web_search(question: str) -> list[SearchResult]:
    """
    Semantic search: Finds most results in the vector DB

    args:
    - question: a question about game industry.
    """
    client = TavilyClient()
    response = client.search(question, num_results=5)

    return [SearchResult(**r) for r in response["results"]]

In [122]:
tavily_search = game_web_search("When was Fifa 98 released?")
print(tavily_search)

[SearchResult(title='FIFA Road to World Cup 98 was released on this day in 1997 🗓️', content='This Is Football Gaming ⚽ (@T_I_F_G). 186 likes 4 replies. FIFA Road to World Cup 98 was released on this day in 1997.', score=0.99968743), SearchResult(title='FIFA Road to World Cup 98 was released on this day in 1997 🗓️', content="FIFA Road to World Cup 98 was released on this day in 1997 🗓️. May be an image of \u200efootball, soccer and \u200etext that says '\u200e.", score=0.9994023), SearchResult(title='FIFA 98: Road to World Cup - LaunchBox Games Database', content='FIFA Road to World Cup 98 is a 1997 soccer game by Electronic Arts released for the Sega Mega Drive as a sequel (or update) to FIFA 97: Gold Edition.', score=0.9991768), SearchResult(title='FIFA Road to World Cup 98 (Mega Drive) - Sega Retro', content='FIFA Road to World Cup 98 is a 1997 soccer game by Electronic Arts released for the Sega Mega Drive as a sequel (or update) to FIFA 97: Gold Edition.', score=0.99906737), Searc

In [ ]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed


In [ ]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes